This notebook retrieves the mining districts found in the mineral deposits [website](https://mrdata.usgs.gov/deposit/). It first pulls the WFS layers found in the API endpoint and retrieves the polygon layer which has the mining districts of interest.

The Discussion page explaining the process taken can be found in the [USMIN Mineral Deposit Database: polygonal features](https://geokb.wikibase.cloud/wiki/Item_talk:Q158190) Item page.

In [1]:
import geopandas as gpd
import pandas as pd
import requests
import fiona
import os
import shapely
from typing import Optional
from fiona import BytesCollection
from shapely.geometry import Polygon
from wbmaker import WikibaseConnection
from dotenv import load_dotenv
from arcgis.gis import GIS

load_dotenv()

fiona.drvsupport.supported_drivers['WFS'] = 'r'

In [2]:
def wfs2gp_df(TYPENAMES, url, bbox=None, wfs_version="2.0.0", outputFormat='application/gml+xml; version=3.2'):
    params = dict(service='WFS', version=wfs_version,request='GetCapabilities', TYPENAMES=TYPENAMES, outputFormat=outputFormat)
    with BytesCollection(requests.get(url,params=params).content) as f:
        df = gpd.GeoDataFrame.from_features(f, crs='EPSG:4326')
    return df

mining_districts = wfs2gp_df('polygons', 'https://mrdata.usgs.gov/services/wfs/deposit',None, '1.1.0', 'json')

In [3]:
# def read_wfs(url : str, params : dict):
#     # params = dict(service='WFS', version=wfs_version,request='GetFeature', TYPENAMES=TYPENAMES, outputFormat=outputFormat)
#     with BytesCollection(requests.get(url,params=params).content) as f:
#         df = gpd.GeoDataFrame.from_features(f, crs='EPSG:4326')
#     return df
# #request=GetFeature&service=WFS&version=1.1.0&TYPENAMES=polygons
# url = 'https://mrdata.usgs.gov/services/wfs/deposit'
# params = {
#     'request':'GetFeature',
#     'service':'WFS',
#     'version':'1.1.0',
#     'typeName':'polygons',
#     'srsName': 'urn:ogc:def:crs:EPSG::4326'
# }
# mining_districts = read_wfs(url, params)

In [4]:
mining_districts.head()

,geometry,gml_id,site_id,ftr_id,ftr_name,last_updt,doi,area_sqkm,area_acres,remarks
0,"POLYGON ((46.74615 -87.88469, 46.74619 -87.884...",polygons.20,MI00001,Mo00363,Eagle East,2017-08-31,10.5066/P9V74HIU,NaN,NaN,NaN
1,"POLYGON ((47.82946 -91.67692, 47.82990 -91.679...",polygons.25,MN00003,Mo00722,Spruce Road,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN
2,"POLYGON ((46.74872 -87.89662, 46.74872 -87.896...",polygons.21,MI00001,Mo00362,Eagle,2017-08-31,10.5066/P9V74HIU,NaN,NaN,NaN
3,"POLYGON ((47.81905 -91.72215, 47.81936 -91.721...",polygons.22,MN00003,Mo00721,Maturi,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN
4,"POLYGON ((47.71916 -91.81861, 47.71935 -91.820...",polygons.23,MN00003,Mo00719,Birch Lake,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN


In [5]:
def reverse_coords(geom: Polygon) -> Polygon:
    coords = geom.exterior.coords.xy
    reverse = tuple(zip(coords[1], coords[0]))
    return Polygon(reverse)

In [6]:
mining_districts['geometry'] = [reverse_coords(x) for x in mining_districts['geometry']]
mining_districts.head()

,geometry,gml_id,site_id,ftr_id,ftr_name,last_updt,doi,area_sqkm,area_acres,remarks
0,"POLYGON ((-87.88469 46.74615, -87.88474 46.746...",polygons.20,MI00001,Mo00363,Eagle East,2017-08-31,10.5066/P9V74HIU,NaN,NaN,NaN
1,"POLYGON ((-91.67692 47.82946, -91.67956 47.829...",polygons.25,MN00003,Mo00722,Spruce Road,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN
2,"POLYGON ((-87.89662 46.74872, -87.89657 46.748...",polygons.21,MI00001,Mo00362,Eagle,2017-08-31,10.5066/P9V74HIU,NaN,NaN,NaN
3,"POLYGON ((-91.72215 47.81905, -91.72119 47.819...",polygons.22,MN00003,Mo00721,Maturi,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN
4,"POLYGON ((-91.81861 47.71916, -91.82029 47.719...",polygons.23,MN00003,Mo00719,Birch Lake,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN


In [7]:
mining_districts = mining_districts.drop_duplicates(subset=['site_id'])


In [8]:
gis = GIS()
counties = gis.content.get('14c5450526a8430298b2fa74da12c2f4')
counties


/tmp/geokb/geokbenv/lib/python3.8/site-packages/arcgis/gis/_impl/_con/_connection.py:296: DeprecationWarning: Using 'method_whitelist' with Retry is deprecated and will be removed in v2.0. Use 'allowed_methods' instead
  max_retries=Retry(
/tmp/geokb/geokbenv/lib/python3.8/site-packages/arcgis/gis/__init__.py:11174: DeprecationWarning: This method will be removed in a future version of Python. Use 'locale.format_string()' instead.
  numViews = locale.format("%d", self.numViews, grouping=True)


<Item title:"USA Census Counties" type:Feature Layer Collection owner:esri_dm>

In [9]:
layer = counties.layers[0]

In [10]:
query_count = layer.query(where="1=1", out_fields=['*'], return_count_only=True)

In [13]:
layer_query = layer.query(
        where="1=1",
        out_fields=['*'],
        as_df=True,
        out_sr=4326,
        # return_all_records=False,
        # result_offset=offset,
        result_record_count=100
    )

In [14]:
layer_query.head()

,COUNTY_FIPS,FIPS,NAME,OBJECTID,POPULATION,POP_SQMI,SHAPE,SQMI,STATE_ABBR,STATE_FIPS,STATE_NAME,Shape__Area,Shape__Length
0,001,01001,Autauga County,1,58805,97.3,"{""rings"": [[[-86.413120727, 32.7073921370001],...",604.37,AL,01,Alabama,0.150256,2.066033
1,003,01003,Baldwin County,2,231767,141.9,"{""rings"": [[[-87.5649079999999, 30.281622], [-...",1633.14,AL,01,Alabama,0.398404,9.305629
2,005,01005,Barbour County,3,25223,27.9,"{""rings"": [[[-85.257838372, 32.147937056], [-8...",904.52,AL,01,Alabama,0.223270,2.695260
3,007,01007,Bibb County,4,22293,35.6,"{""rings"": [[[-87.06574294, 33.2469132270001], ...",626.17,AL,01,Alabama,0.156473,1.887519
4,009,01009,Blount County,5,59134,90.9,"{""rings"": [[[-86.453024823, 34.259323463], [-8...",650.63,AL,01,Alabama,0.164405,2.423466


In [15]:
from shapely import wkt
mining_districts['centroid'] = [wkt.loads(str(x)).centroid for x in mining_districts['geometry']]
mining_districts.head()

,geometry,gml_id,site_id,ftr_id,ftr_name,last_updt,doi,area_sqkm,area_acres,remarks,centroid
0,"POLYGON ((-87.88469 46.74615, -87.88474 46.746...",polygons.20,MI00001,Mo00363,Eagle East,2017-08-31,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-87.8802497576474 46.746424629721176)
1,"POLYGON ((-91.67692 47.82946, -91.67956 47.829...",polygons.25,MN00003,Mo00722,Spruce Road,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-91.67306264135848 47.83455981687981)
5,"POLYGON ((-91.95769 47.62485, -91.95477 47.625...",polygons.24,MN00002,Mo00746,NorthMet,2018-04-16,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-91.97153846066311 47.61665047292834)
7,"POLYGON ((-90.11476 37.61021, -90.11857 37.528...",polygons.27,MO00001,Mr00036,Fredericktown District,2018-06-07,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-90.2061544546558 37.57042461027146)
9,"POLYGON ((-76.41196 40.26707, -76.41213 40.267...",polygons.41,PA00001,Mo00749,Cornwall,2018-04-18,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-76.4122036035501 40.26710794585181)


In [16]:
mining_districts['centroid_lon'] = [row.x for row in mining_districts['centroid']]
mining_districts['centroid_lat'] = [row.y for row in mining_districts['centroid']]
mining_districts.head()

,geometry,gml_id,site_id,ftr_id,ftr_name,last_updt,doi,area_sqkm,area_acres,remarks,centroid,centroid_lon,centroid_lat
0,"POLYGON ((-87.88469 46.74615, -87.88474 46.746...",polygons.20,MI00001,Mo00363,Eagle East,2017-08-31,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-87.8802497576474 46.746424629721176),-87.880250,46.746425
1,"POLYGON ((-91.67692 47.82946, -91.67956 47.829...",polygons.25,MN00003,Mo00722,Spruce Road,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-91.67306264135848 47.83455981687981),-91.673063,47.834560
5,"POLYGON ((-91.95769 47.62485, -91.95477 47.625...",polygons.24,MN00002,Mo00746,NorthMet,2018-04-16,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-91.97153846066311 47.61665047292834),-91.971538,47.616650
7,"POLYGON ((-90.11476 37.61021, -90.11857 37.528...",polygons.27,MO00001,Mr00036,Fredericktown District,2018-06-07,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-90.2061544546558 37.57042461027146),-90.206154,37.570425
9,"POLYGON ((-76.41196 40.26707, -76.41213 40.267...",polygons.41,PA00001,Mo00749,Cornwall,2018-04-18,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-76.4122036035501 40.26710794585181),-76.412204,40.267108


In [17]:
def fix_label(label: str) -> str:
    if 'Mining District' not in label:
        return label  + ' Mining District'
    if 'District' in label and not 'Mining District' in label:
        return label.replace('District','Mining District')
    return label

labels = [ fix_label(x) for x in mining_districts['ftr_name'] ]
aliases = [ x if 'mining district' not in x.lower() else '' for x in mining_districts['ftr_name'] ]

In [18]:
mining_districts['label'] = labels
mining_districts['aliases'] = aliases

In [19]:
# make sure all shapes are valid before comparing
layer_query['SHAPE'] = [shapely.make_valid(x.as_shapely) if not shapely.is_valid(x.as_shapely) else x.as_shapely for x in layer_query['SHAPE']]
mining_districts['geometry'] = [shapely.make_valid(x) if not shapely.is_valid(x) else x for x in mining_districts['geometry']]

In [20]:
def match_shapes(poly: Polygon, rows: pd.DataFrame) -> tuple:
    if not shapely.is_valid(poly):
        poly = shapely.make_valid(poly)
    for i in range(len(rows)):
        row = rows.iloc[i]
        multipoly = row['SHAPE']
        if poly.intersects(multipoly):
            return row['STATE_NAME'], row['NAME']
    return '', ''

In [21]:
county_state = []
for i in range(len(mining_districts)):
    district = mining_districts.iloc[i]
    ct_st_match = match_shapes(district['geometry'], layer_query)
    # if either state or county is '', then match not found
    if '' in ct_st_match:
        print('Shape Match not found for provided Polygon\n',f'District not matched: {district["ftr_name"]}')
    county_state.append(ct_st_match)

In [22]:
mining_districts['state'] = [x[0] for x in county_state]
mining_districts['county'] = [x[1] for x in county_state]

In [23]:
mining_districts.head()

,geometry,gml_id,site_id,ftr_id,ftr_name,last_updt,doi,area_sqkm,area_acres,remarks,centroid,centroid_lon,centroid_lat,label,aliases,state,county
0,"POLYGON ((-87.88469 46.74615, -87.88474 46.746...",polygons.20,MI00001,Mo00363,Eagle East,2017-08-31,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-87.8802497576474 46.746424629721176),-87.880250,46.746425,Eagle East Mining District,Eagle East,Michigan,Marquette County
1,"POLYGON ((-91.67692 47.82946, -91.67956 47.829...",polygons.25,MN00003,Mo00722,Spruce Road,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-91.67306264135848 47.83455981687981),-91.673063,47.834560,Spruce Road Mining District,Spruce Road,Minnesota,Lake County
5,"POLYGON ((-91.95769 47.62485, -91.95477 47.625...",polygons.24,MN00002,Mo00746,NorthMet,2018-04-16,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-91.97153846066311 47.61665047292834),-91.971538,47.616650,NorthMet Mining District,NorthMet,Minnesota,St. Louis County
7,"POLYGON ((-90.11476 37.61021, -90.11857 37.528...",polygons.27,MO00001,Mr00036,Fredericktown District,2018-06-07,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-90.2061544546558 37.57042461027146),-90.206154,37.570425,Fredericktown District Mining District,Fredericktown District,Missouri,Bollinger County
9,"POLYGON ((-76.41196 40.26707, -76.41213 40.267...",polygons.41,PA00001,Mo00749,Cornwall,2018-04-18,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-76.4122036035501 40.26710794585181),-76.412204,40.267108,Cornwall Mining District,Cornwall,Pennsylvania,Lebanon County


In [24]:
import re
def check_for_format_error(districts_df: gpd.GeoDataFrame) -> list:
    # return any dates that are not in YYYY-MM-DD format
    return [x for x in districts_df['last_updt'] if not re.search(r'\d\d\d\d-\d\d-\d\d', x)]

check_for_format_error(mining_districts)

['2018-3-16']

In [25]:
import datetime
def fix_time(t:str, t_format: str) -> str:
    # converts to YYYY-MM-DD format
    d = datetime.datetime.strptime(t, t_format)
    return d.strftime('%Y-%m-%d')

In [26]:
new_time = [fix_time(x, '%Y-%m-%d') if not re.search(r'\d\d\d\d-\d\d-\d\d', x) else x for x in mining_districts['last_updt']]
mining_districts['last_updt'] = new_time
# verify that date format in last_updt column has been fixed
check_for_format_error(mining_districts)

[]

In [27]:
today = datetime.date.today()
mining_districts['date_processed'] = today.isoformat()

In [28]:
mining_districts.head()

,geometry,gml_id,site_id,ftr_id,ftr_name,last_updt,doi,area_sqkm,area_acres,remarks,centroid,centroid_lon,centroid_lat,label,aliases,state,county,date_processed
0,"POLYGON ((-87.88469 46.74615, -87.88474 46.746...",polygons.20,MI00001,Mo00363,Eagle East,2017-08-31,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-87.8802497576474 46.746424629721176),-87.880250,46.746425,Eagle East Mining District,Eagle East,Michigan,Marquette County,2023-08-29
1,"POLYGON ((-91.67692 47.82946, -91.67956 47.829...",polygons.25,MN00003,Mo00722,Spruce Road,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-91.67306264135848 47.83455981687981),-91.673063,47.834560,Spruce Road Mining District,Spruce Road,Minnesota,Lake County,2023-08-29
5,"POLYGON ((-91.95769 47.62485, -91.95477 47.625...",polygons.24,MN00002,Mo00746,NorthMet,2018-04-16,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-91.97153846066311 47.61665047292834),-91.971538,47.616650,NorthMet Mining District,NorthMet,Minnesota,St. Louis County,2023-08-29
7,"POLYGON ((-90.11476 37.61021, -90.11857 37.528...",polygons.27,MO00001,Mr00036,Fredericktown District,2018-06-07,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-90.2061544546558 37.57042461027146),-90.206154,37.570425,Fredericktown District Mining District,Fredericktown District,Missouri,Bollinger County,2023-08-29
9,"POLYGON ((-76.41196 40.26707, -76.41213 40.267...",polygons.41,PA00001,Mo00749,Cornwall,2018-04-18,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-76.4122036035501 40.26710794585181),-76.412204,40.267108,Cornwall Mining District,Cornwall,Pennsylvania,Lebanon County,2023-08-29


In [29]:
name = 'GEOKB_CLOUD'
geokb = WikibaseConnection(name)

In [30]:
def item_search(label: str, instance_of: str, bot_name: str) -> str:
  sparql_endpoint = os.environ[f'WB_SPARQL_{bot_name}']
  query = f'''PREFIX wdt: <https://geokb.wikibase.cloud/prop/direct/>
  PREFIX wd:  <https://geokb.wikibase.cloud/entity/>
  SELECT ?item
  WHERE {{
    ?item rdfs:label ?label ;
       wdt:P1 wd:{instance_of} .
    FILTER CONTAINS( LCASE(?label), "{label.lower()}") .

    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en" . }}
  }}
  '''

  params = {
      'query': query,
      'format': 'json'
  }

  res = requests.get(sparql_endpoint, params=params, timeout=100)
  json_res =res.json()
  item_result = (json_res['results']['bindings'][0]['item']['value']
                  if 'results' in json_res
                  and len(json_res['results']['bindings']) > 0
                  and 'item' in json_res['results']['bindings'][0]
                  else None)
  return item_result.split('/')[-1] if item_result is not None else None

In [31]:
county_instance_of = 'Q481'
mining_districts['county'].replace(r'St\.', 'Saint',regex=True, inplace=True)
county_items = [
    item_search(f'{county}, {state}', county_instance_of, name) 
    for (county,state) 
    in zip(mining_districts['county'], mining_districts['state'])
]
mining_districts['county_qid'] = county_items

In [32]:
state_instance_of = 'Q229'
state_items = [ item_search(state, state_instance_of, name) for state in mining_districts['state'] ]
mining_districts['state_qid'] = state_items
mining_districts.head()

,geometry,gml_id,site_id,ftr_id,ftr_name,last_updt,doi,area_sqkm,area_acres,remarks,centroid,centroid_lon,centroid_lat,label,aliases,state,county,date_processed,county_qid,state_qid
0,"POLYGON ((-87.88469 46.74615, -87.88474 46.746...",polygons.20,MI00001,Mo00363,Eagle East,2017-08-31,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-87.8802497576474 46.746424629721176),-87.880250,46.746425,Eagle East Mining District,Eagle East,Michigan,Marquette County,2023-08-29,Q51002,Q230
1,"POLYGON ((-91.67692 47.82946, -91.67956 47.829...",polygons.25,MN00003,Mo00722,Spruce Road,2018-04-10,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-91.67306264135848 47.83455981687981),-91.673063,47.834560,Spruce Road Mining District,Spruce Road,Minnesota,Lake County,2023-08-29,Q51956,Q239
5,"POLYGON ((-91.95769 47.62485, -91.95477 47.625...",polygons.24,MN00002,Mo00746,NorthMet,2018-04-16,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-91.97153846066311 47.61665047292834),-91.971538,47.616650,NorthMet Mining District,NorthMet,Minnesota,Saint Louis County,2023-08-29,Q51975,Q239
7,"POLYGON ((-90.11476 37.61021, -90.11857 37.528...",polygons.27,MO00001,Mr00036,Fredericktown District,2018-06-07,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-90.2061544546558 37.57042461027146),-90.206154,37.570425,Fredericktown District Mining District,Fredericktown District,Missouri,Bollinger County,2023-08-29,Q52056,Q244
9,"POLYGON ((-76.41196 40.26707, -76.41213 40.267...",polygons.41,PA00001,Mo00749,Cornwall,2018-04-18,10.5066/P9V74HIU,NaN,NaN,NaN,POINT (-76.4122036035501 40.26710794585181),-76.412204,40.267108,Cornwall Mining District,Cornwall,Pennsylvania,Lebanon County,2023-08-29,Q53874,Q261


In [33]:
def add_new_item(cols:dict, instance_of_val: str, qid: Optional[str]) -> None:
    keep = geokb.action_if_exists.KEEP
    aor = geokb.action_if_exists.APPEND_OR_REPLACE
    replace = geokb.action_if_exists.REPLACE_ALL
    
    references = geokb.models.References()
    districts_ref = geokb.datatypes.URL(
        prop_nr=geokb.prop_lookup['reference URL'],
        value='https://mrdata.usgs.gov/services/wfs/deposit?request=GetCapabilities&service=WFS&version=1.1.0',
    )
    updt_ref = geokb.datatypes.Time(
        prop_nr=geokb.prop_lookup['last update'],
        time=cols['last_updt']+'T00:00:00Z',
    )
    references.add(districts_ref)
    references.add(updt_ref)

    if qid is None:
        item = geokb.wbi.item.new()
    else:
        item = geokb.wbi.item.get(qid)
    item.labels.set('en', cols['label'], action_if_exists=replace)
    item.descriptions.set(
        'en', 
        f'Mining district found in {cols["county"]}, {cols["state"]}',
        action_if_exists=replace
    )
    item.claims.add(
        geokb.datatypes.Item(
                prop_nr=geokb.prop_lookup["instance of"],
                value=instance_of_val,
                references=references
        ),
        action_if_exists=aor
    )
    item.claims.add(
        geokb.datatypes.ExternalID(
            prop_nr=geokb.prop_lookup["GML ID"],
            value=cols['gml_id'],
            references=references
        ),
        action_if_exists=aor
    )
    item.claims.add(
        geokb.datatypes.ExternalID(
            prop_nr=geokb.prop_lookup["Feature ID"],
            value=cols['ftr_id'],
            references=references
        ),
        action_if_exists=aor
    )
    item.claims.add(
        geokb.datatypes.ExternalID(
            prop_nr=geokb.prop_lookup["Site ID"],
            value=cols['site_id'],
            references=references
        ),
        action_if_exists=aor
    )
    item.claims.add(
        geokb.datatypes.ExternalID(
            prop_nr=geokb.prop_lookup["DOI"],
            value=cols['doi'],
            references=references
        ),
        action_if_exists=aor
    )
    item.claims.add(
        geokb.datatypes.Item(
            prop_nr=geokb.prop_lookup["U.S. county"],
            value=cols['county_qid'],
            references=references
        ),
        action_if_exists=aor
    )
    item.claims.add(
        geokb.datatypes.Item(
            prop_nr=geokb.prop_lookup["U.S. state"],
            value=cols['state_qid'],
            references=references
        ),
        action_if_exists=aor
    )
    item.claims.add( geokb.datatypes.Time(
            prop_nr=geokb.prop_lookup['retrieved'],
            time=cols['date_processed']+'T00:00:00Z',
        ),
        action_if_exists=replace
    )
    item.claims.add( geokb.datatypes.GlobeCoordinate(
            prop_nr=geokb.prop_lookup['coordinate location'],
            latitude=cols['centroid_lat'],
            longitude=cols['centroid_lon'],
            references=references
        ),
        action_if_exists=aor
    )
    response = item.write(
        summary="Added mining districts"
    )
    print(cols['label'], response.id)

In [34]:
min_dist_instance_of = 'Q55213'

for i in range(len(mining_districts)):
    fields = mining_districts.iloc[i]
    qid = item_search(fields['label'], min_dist_instance_of, name) or item_search(fields['ftr_name'], min_dist_instance_of, name)
    add_new_item(fields, min_dist_instance_of, qid)
    break

Eagle East Mining District Q157855
